采用BERT模型预训练模型生成文本的词向量表示

In [1]:
# 步骤1：导入必要的库
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

# 步骤2：加载预训练BERT模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [2]:
# 文本准备
import pandas as pd
df = pd.read_csv("data_syn.csv",encoding='utf-8')
texts = df['text'].astype(str).tolist() # 转换为Python列表
print(texts) # 查看准备的文本
# 文本分词
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
print(encoded_input) # 查看文本分词后的结果

['五点 十多 去 店里 都 没开 灯 很 暗 真的 很 影响 用餐 体验 南昌 吃 感觉 完全 不同 我用 会员卡 居然 说 没 注册 完全 说 没查 再 报 一遍 号码 都 无所谓 纸质 菜单 单 去 买单 没 下 餐巾纸 水饺 就算 居然 还 说 是不是 没 下单 店里 调料 都 快没了 不加 银耳 没 不 明明 两个 锅 只加 锅里 锅 干干净净 放在 不加 吃 完 银耳 锅 不加 银耳 不 开灯 小灯 吊灯 都 看不清 菜 怪不得 这家 店 没什么 人 吃饭', '朋友 带来 吃 说 霸占 南昌 平价 火锅 悠闲 味道 确实 不赖 四个 女生 加 三次 菜 三份 鸭血 真的 他家 鸭 血下 辣锅 好 美味 惊叹 蛙 蛙 美味 贡菜 美味 肉肉 美味 冰激凌 美味 反而 大家 都 很 好奇 鹿肉 却 没 吃 出来 不', '美味 服务 好 经常 带 小孩 一家人 去 吃', '南昌 几天 经常 大街 上 看到 季季红 广告牌 感觉 应该 本地 比较 有名 火锅 尝尝 薄荷 环境 店面 挺 干净 人 不 服务 铃 服务 扫码 点餐 结账 后 有人 送餐 菜品 价格 不高 两个 人元 吃 饱饱 调味 台 调料 挺 银耳汤 门口 冰激凌 挖 菜品 口味 还 中规中矩 总体 还', '季季红 火锅 食材 新鲜 味道 正宗 锅底 浓郁 调味 恰到好处 服务周到 热情 火锅 爱好者 不错 选择', '非常 美味 真的 推荐 吃 都 美味 服务 好 态度 已经 不是 一次 两次 吃 超 喜欢 季季红 火锅', '感觉 火锅 味道 挺不错 服务员 挺 耐心 锅底 差不多 没有 汤时 会 主动 请 是否 需要 加汤', '菜 很 新鲜 锅底 很 好 个人 很 推荐', '特别 美味 环境 很 好 一家人 都 特别 爱 吃 主要 服务 下次 还会 再', '服务员 态度 非常 热情 刚来 会 主动 请问 是否 需要 围裙', '服务态度 很 好 价格 优惠 值得 推荐 个人 比较 爱 吃 肉 卷 土豆 粉粉 好 美味', '「 鲜嫩 肥牛 卷 性价比 服务 环境 口味', '整体 感觉 很 好 很 氛围 感 上菜 速度 一下子 上 齐 服务 加汤 很 主动 价格 经济 实惠 美味', '看 众人 一水 好评 不禁 人 好奇 店能 人 流连忘返 带 家人 尝 名不虚传 主要 上菜

In [3]:
# 使用BERT模型生成词向量
import os
import pandas as pd
import numpy as np
# 定义批次大小
batch_size = 8
# 统一所有向量的长度到512
uniform_length = 512  

# 分批预处理文本
def batch_encode(texts, batch_size):
    for i in range(0, len(texts), batch_size):
        yield tokenizer(texts[i:i+batch_size], padding=True, truncation=True, max_length=512, return_tensors="tf")

# 创建一个目录来保存词向量
if not os.path.exists('bert_vectors'):
    os.makedirs('bert_vectors')

def process_vector(vector, max_length):
    """确保向量的长度等于max_length."""
    if len(vector) > max_length:
        return vector[:max_length]  # 截断多余的部分
    elif len(vector) < max_length:
        padding = np.zeros((max_length - len(vector), vector.shape[1]), dtype=np.float32)
        return np.vstack((vector, padding))  # 填充到max_length
    return vector

# 分批处理并保存结果
for i, batch in enumerate(batch_encode(texts, batch_size)):
    try:
        outputs = model(batch)
        last_hidden_states = outputs.last_hidden_state.numpy()  # 转换成NumPy数组以便保存
        
        # 确保所有向量的长度一致
        processed_vectors = np.array([process_vector(vec, uniform_length) for vec in last_hidden_states])
        
        # 保存处理后的向量
        file_name = f'bert_vectors/batch_{i:04d}.npy'  # 格式化文件名以保持排序
        with open(file_name, 'wb') as f:
            np.save(f, processed_vectors)
    
    except tf.errors.ResourceExhaustedError as e:
        print(f"资源不足，无法处理批次 {i}：{e}")

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

# 初始化分词器和模型
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = TFBertModel.from_pretrained('bert-base-chinese')

# 读取数据
df = pd.read_csv("data_syn.csv", encoding='utf-8')
texts = df['text'].astype(str).tolist()
print("Texts prepared:", texts)

# 定义分批预处理函数
def batch_encode(texts, batch_size, max_length=512):
    for i in range(0, len(texts), batch_size):
        yield tokenizer(texts[i:i+batch_size], padding='max_length', truncation=True, max_length=max_length, return_tensors="tf")

# 创建存储目录
vector_dir = 'bert_vectors'
if not os.path.exists(vector_dir):
    os.makedirs(vector_dir)

# 分批处理文本，生成并保存向量
for i, batch in enumerate(batch_encode(texts, batch_size=8)):
    outputs = model(batch['input_ids'], attention_mask=batch['attention_mask'])
    last_hidden_states = outputs.last_hidden_state.numpy()  # 获得BERT的输出

    # 保存向量
    file_name = os.path.join(vector_dir, f'batch_{i:04d}.npy')
    with open(file_name, 'wb') as f:
        np.save(f, last_hidden_states)
    print(f"Batch {i} processed and saved to {file_name}.")

print("All batches processed and saved successfully.")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Texts prepared: ['五点 十多 去 店里 都 没开 灯 很 暗 真的 很 影响 用餐 体验 南昌 吃 感觉 完全 不同 我用 会员卡 居然 说 没 注册 完全 说 没查 再 报 一遍 号码 都 无所谓 纸质 菜单 单 去 买单 没 下 餐巾纸 水饺 就算 居然 还 说 是不是 没 下单 店里 调料 都 快没了 不加 银耳 没 不 明明 两个 锅 只加 锅里 锅 干干净净 放在 不加 吃 完 银耳 锅 不加 银耳 不 开灯 小灯 吊灯 都 看不清 菜 怪不得 这家 店 没什么 人 吃饭', '朋友 带来 吃 说 霸占 南昌 平价 火锅 悠闲 味道 确实 不赖 四个 女生 加 三次 菜 三份 鸭血 真的 他家 鸭 血下 辣锅 好 美味 惊叹 蛙 蛙 美味 贡菜 美味 肉肉 美味 冰激凌 美味 反而 大家 都 很 好奇 鹿肉 却 没 吃 出来 不', '美味 服务 好 经常 带 小孩 一家人 去 吃', '南昌 几天 经常 大街 上 看到 季季红 广告牌 感觉 应该 本地 比较 有名 火锅 尝尝 薄荷 环境 店面 挺 干净 人 不 服务 铃 服务 扫码 点餐 结账 后 有人 送餐 菜品 价格 不高 两个 人元 吃 饱饱 调味 台 调料 挺 银耳汤 门口 冰激凌 挖 菜品 口味 还 中规中矩 总体 还', '季季红 火锅 食材 新鲜 味道 正宗 锅底 浓郁 调味 恰到好处 服务周到 热情 火锅 爱好者 不错 选择', '非常 美味 真的 推荐 吃 都 美味 服务 好 态度 已经 不是 一次 两次 吃 超 喜欢 季季红 火锅', '感觉 火锅 味道 挺不错 服务员 挺 耐心 锅底 差不多 没有 汤时 会 主动 请 是否 需要 加汤', '菜 很 新鲜 锅底 很 好 个人 很 推荐', '特别 美味 环境 很 好 一家人 都 特别 爱 吃 主要 服务 下次 还会 再', '服务员 态度 非常 热情 刚来 会 主动 请问 是否 需要 围裙', '服务态度 很 好 价格 优惠 值得 推荐 个人 比较 爱 吃 肉 卷 土豆 粉粉 好 美味', '「 鲜嫩 肥牛 卷 性价比 服务 环境 口味', '整体 感觉 很 好 很 氛围 感 上菜 速度 一下子 上 齐 服务 加汤 很 主动 价格 经济 实惠 美味', '看 众人 一水 好评 不禁 人 好奇 店能 人 流连忘返 带